### Introduction to Trajectory Optimization
- Introduce dynamics equations As + Bu = z
- Single obstacle encoding as constraints
- Speed limit

#### Introduce simple problem
- single vehicle, treated as a point
- single obstacle
- start & end goal
- objective function: minimize time

### Space Station Problem Introduction
- Go thru same steps as above
- Intro multi-vehicle problem
    - Collision avoidance
    - Each waypoint visited by only one vehicle

### Encoding of Problem - MIQCP